In [1]:
import pandas as pd
import numpy as np
import wfdb
import ast

def load_raw_data(df, sampling_rate):
    if sampling_rate == 100:
        data = [wfdb.rdsamp(f) for f in df.filename_lr]
    else:
        data = [wfdb.rdsamp(f) for f in df.filename_hr]
    data = np.array([signal for signal, meta in data])
    return data

sampling_rate=100

# load and convert annotation data
df = pd.read_csv('ptbxl_database.csv', index_col='ecg_id')
df.scp_codes = df.scp_codes.apply(lambda x: ast.literal_eval(x))

# Load raw signal data
Signals = load_raw_data(df, sampling_rate)

# Load scp_statements.csv for diagnostic aggregation
agg_df = pd.read_csv('scp_statements.csv', index_col=0)
agg_df = agg_df[agg_df.diagnostic == 1]

def aggregate_diagnostic(y_dic):
    tmp = []
    for key in y_dic.keys():
        if key in agg_df.index:
            tmp.append(agg_df.loc[key].diagnostic_class)
    return list(set(tmp))

# Apply diagnostic superclass
df.reset_index(inplace=True)
Labels = df.scp_codes.apply(aggregate_diagnostic)

In [2]:
y=[]
x=[]
for i in range(len(Labels)):
    if Labels[i] == list(['CD']):
        y.append('CD')
        x.append(Signals[i,:,0])
    if Labels[i] == list(['HYP']):
        y.append('HYP')
        x.append(Signals[i,:,0])
    if Labels[i] == list(['NORM']):
        y.append('NORM')
        x.append(Signals[i,:,0])
    if Labels[i] == list(['STTC']):
        y.append('STTC')
        x.append(Signals[i,:,0])
    if Labels[i] == list(['MI']):
        y.append('MI')
        x.append(Signals[i,:,0])

x = np.array(x)
y = np.array(y)

In [25]:
augmented_signals = []
augmented_labels = []


for label in ['CD', 'HYP', 'MI', 'STTC']:
    target_indices = np.where(y == label)[0]
    augmentation_factor = (np.count_nonzero(y == 'NORM')  // np.count_nonzero(y == label)) -1
    
    for index in target_indices:
        repeated_signal = np.tile(x[index], (augmentation_factor, 1))
        noisy_signal = repeated_signal + 0.01 * np.random.randn(*repeated_signal.shape)
        repeated_signal_2d = repeated_signal.reshape(-1, repeated_signal.shape[-1])
        augmented_signals.append(noisy_signal)
        augmented_labels.extend([y[index]] * augmentation_factor)
    
X = np.concatenate([x] + augmented_signals)
Y = np.concatenate([y, np.array(augmented_labels)])

In [37]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

label_encoder = LabelEncoder()
Y_encoded = label_encoder.fit_transform(Y)

Y_onehot = to_categorical(Y_encoded, num_classes=5)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y_onehot, test_size=0.2, random_state=None)

In [31]:
from tensorflow.keras import layers, models

def build_1d_resnet18(input_shape, num_classes):
    input_tensor = layers.Input(shape=input_shape)

    # Initial Convolution
    x = layers.Conv1D(64, kernel_size=7, strides=2, padding='same', activation='relu')(input_tensor)
    x = layers.MaxPooling1D(pool_size=3, strides=2, padding='same')(x)

    # Residual Blocks
    x = residual_block_1d(x, 64, 1)
    x = residual_block_1d(x, 128, 2)
    x = residual_block_1d(x, 256, 2)
    x = residual_block_1d(x, 512, 2)

    # Global Average Pooling
    x = layers.GlobalAveragePooling1D()(x)

    # Fully Connected layer
    x = layers.Dense(num_classes, activation='softmax')(x)

    # Create model
    model = models.Model(inputs=input_tensor, outputs=x, name='resnet18_1d')

    return model

def residual_block_1d(input_tensor, filters, strides):
    shortcut = input_tensor

    # First convolution layer
    x = layers.Conv1D(filters, kernel_size=3, strides=strides, padding='same', activation='relu')(input_tensor)

    # Second convolution layer
    x = layers.Conv1D(filters, kernel_size=3, padding='same', activation='relu')(x)

    # Shortcut connection if needed
    if strides != 1 or input_tensor.shape[-1] != filters:
        shortcut = layers.Conv1D(filters, kernel_size=1, strides=strides, padding='valid', activation='relu')(input_tensor)

    # Add shortcut to main path
    x = layers.add([x, shortcut])

    return x

In [32]:
input_shape = (1000, 12)
num_classes = 5  

resnet18_1d_model = build_1d_resnet18(input_shape, num_classes)

resnet18_1d_model.summary()

2024-01-23 21:54:28.254639: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-01-23 21:54:28.254711: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-01-23 21:54:28.254721: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-01-23 21:54:28.255325: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-01-23 21:54:28.255848: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "resnet18_1d"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 1000, 12)]           0         []                            
                                                                                                  
 conv1d (Conv1D)             (None, 500, 64)              5440      ['input_1[0][0]']             
                                                                                                  
 max_pooling1d (MaxPooling1  (None, 250, 64)              0         ['conv1d[0][0]']              
 D)                                                                                               
                                                                                                  
 conv1d_1 (Conv1D)           (None, 250, 64)              12352     ['max_pooling1d[0][0

In [33]:
from tensorflow.keras.optimizers.legacy import Adam

optimizer = Adam(learning_rate=0.001)
resnet18_1d_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [34]:
from sklearn.model_selection import StratifiedKFold
k_folds = 5
skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

In [47]:
# Initialize lists to store the results
losses = []
accuracies = []

# Iterate over the folds
for train_index, test_index in skf.split(X_train, Y_train[:,0]):
    x_train, x_val = X_train[train_index], X_train[test_index]
    y_train, y_val = Y_train[train_index], Y_train[test_index]
    
    resnet18_1d_model.compile(
        optimizer=optimizer, 
        loss='categorical_crossentropy', 
        metrics=['accuracy'])
    
    # Train the model
    history = resnet18_1d_model.fit(
        x_train, y_train, 
        epochs=10, 
        batch_size=32, 
        validation_data=(x_val, y_val), 
        verbose=0)

    # Evaluate the model on the validation set
    loss, accuracy = resnet18_1d_model.evaluate(x_val, y_val, verbose=0)

    # Store the results
    losses.append(loss)
    accuracies.append(accuracy)

# Print the average results over all folds
print(f'Average Loss: {np.mean(losses)}')
print(f'Average Accuracy: {np.mean(accuracies)}')

ValueError: in user code:

    File "/Users/harshbalgude/miniconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/Users/harshbalgude/miniconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/harshbalgude/miniconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/Users/harshbalgude/miniconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "/Users/harshbalgude/miniconda3/lib/python3.11/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/harshbalgude/miniconda3/lib/python3.11/site-package

In [48]:
# Iterate over the folds
for train_index, test_index in skf.split(X_train, Y_train[:, 0]):
    x_train, x_val = X_train[train_index], X_train[test_index]
    y_train, y_val = Y_train[train_index], Y_train[test_index]

    # Ensure x_train and x_val have the correct shape
    # The shape should be (batch_size, time_steps, features)
    x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
    x_val = x_val.reshape((x_val.shape[0], x_val.shape[1], 1))

    resnet18_1d_model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy'])

    # Train the model
    history = resnet18_1d_model.fit(
        x_train, y_train,
        epochs=10,
        batch_size=32,
        validation_data=(x_val, y_val),
        verbose=0)

    # Evaluate the model on the validation set
    loss, accuracy = resnet18_1d_model.evaluate(x_val, y_val, verbose=0)

    # Store the results
    losses.append(loss)
    accuracies.append(accuracy)


ValueError: in user code:

    File "/Users/harshbalgude/miniconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/Users/harshbalgude/miniconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/harshbalgude/miniconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/Users/harshbalgude/miniconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "/Users/harshbalgude/miniconda3/lib/python3.11/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/harshbalgude/miniconda3/lib/python3.11/site-package